In [1]:
##%load schools.md
from IPython.display import Markdown, display

display(Markdown("schools.md"))

## [MA Department of Elementary and Secondary Education](https://profiles.doe.mass.edu/)

A. School profiles using [statewide](https://profiles.doe.mass.edu/statereport/) reports detailing:
   
1. [Student Enrollment](https://profiles.doe.mass.edu/statereport/enrollmentbygrade.aspx)
    * [public](https://profiles.doe.mass.edu/statereport/enrollmentbygrade.aspx) - enrollment  
    * [private](https://profiles.doe.mass.edu/statereport/nonpublicschoolreport.aspx) - enrollment  
2. [Assessments](https://profiles.doe.mass.edu/statereport/nextgenmcas.aspx) - mcas  
3. [Finances](https://profiles.doe.mass.edu/statereport/ppx.aspx)
    * [Per pupil expenditures](https://profiles.doe.mass.edu/statereport/ppx.aspx) - ppx  
    * [Chapter 70 Funding/Spending](https://profiles.doe.mass.edu/statereport/netschoolspendingtrend.aspx) - nss  
    * [salaries](https://profiles.doe.mass.edu/statereport/teachersalaries.aspx) - teacher_salaries  
4. [Teachers](https://profiles.doe.mass.edu/statereport/teacherbyracegender.aspx)   
    * [race/gender](https://profiles.doe.mass.edu/statereport/teacherbyracegender.aspx) - teacher_race  
    * [age](https://profiles.doe.mass.edu/statereport/agestaffing.aspx) - teacher_age  
    * [program area](https://profiles.doe.mass.edu/statereport/programareastaffing.aspx) - teacher_program_area  


1. Common [postgres](https://www.postgresql.org/) database tables prefixed with schools_ under infrastructure schema; e.g. ```infrastructure.schools_enrollment```

    * typical table definition
        ```
                CREATE TABLE infrastructure.schools_{category} (
                    "school_id" INTEGER,
                    "year" SMALLINT,
                    "{category}" SMALLINT,
                    "value" INT,
                    PRIMARY KEY ("school_id","year","{category}")
                );
                CREATE INDEX schools_{category}_idx 
                    ON infrastructure.schools_{category}("school_id");
                CREATE INDEX schools_{category}_year_idx 
                    ON infrastructure.schools_{category}("year");
        ```
  
    * ```schools_mcas``` table adds grade, subject (math/ela)
  
2. columns melted into ```category```.  key-value pairs are defined in ```common.int_value_pairs```

    * school_id - names
    * school_type - public, private, district, charter and collaborative 
    * category labels; e.g. teacher_race e.g. 'Male', 'Female', 'White'
  
  
3. Extract, Transform and Load

    * Run time about 5 minutes
    * space requirements
        * enrollments 600K rows / 26M size
        * mcas        1.6M rows / 70M size
        * nss/ppx      60K rows /  3M size
        * teachers    150K rows /  7M size

## Set-Up

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv (
        find_dotenv (
            usecwd=True
        ),
    override=True
) # read local .env file and override any existing

from sqlalchemy import create_engine
from os import environ

username     =  environ.get("POSTGRES_USERNAME", "postgres")
password     =  environ.get("POSTGRES_PASSWORD", "postgres")
ipaddress    =  environ.get("POSTGRES_IPADDRESS", "localhost")
port         =  environ.get("POSTGRES_PORT", "5432")
dbname       =  environ.get("POSTGRES_DBNAME", "MA")

#establish database connection for Transform queries and Loads
cnx= create_engine(f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')


In [ ]:
from DESE_selenium import *

## Extract & Load

In [ ]:
stub = 'https://profiles.doe.mass.edu/statereport/'

#### Enrollments

In [ ]:
##non-public schools
url = 'nonpublicschoolreport.aspx'
driver = start_up(stub+url)
time.sleep(2)

data = get_private_schools_enrollments(driver)
driver.close()

enrollments, int_value_pairs, school_id_location = \
    transform_private_schools_enrollments(data)

cnx.execute(table_create_schools_enrollment)

enrollments.to_sql('schools_enrollment',schema='infrastructure',
                    con=cnx,if_exists='append',index=False) 
int_value_pairs[~int_value_pairs.duplicated(['key','item'])].to_sql('int_value_pairs',schema='common',
                    con=cnx,if_exists='append',index=False)

##public schools

url = 'enrollmentbygrade.aspx'
driver = start_up(stub+url)
time.sleep(2)

data = get_schools_enrollments(driver)
driver.close()

enrollments, int_value_pairs = transform_enrollments(data)

enrollments.to_sql('schools_enrollment',schema='infrastructure',
                    con=cnx,if_exists='append',index=False) 

int_value_pairs[~int_value_pairs.duplicated(['key','item'])].to_sql('int_value_pairs',schema='common',
                    con=cnx,if_exists='append',index=False)


##NB common schema instead of infrastructure
cnx.execute(table_create_school_id_location)
school_id_location.rename(columns={'value':'dor'})\
        .to_sql('school_id_location',schema='common',
                con=cnx,if_exists='append',index=False)

#### MCAS performance

In [ ]:
url = 'nextgenmcas.aspx'

driver = start_up(stub + url)
time.sleep(2)

data = get_mcas(driver)
driver.close()
mcas, int_value_pairs = transform_mcas(data.copy())

cnx.execute(table_create_mcas)
mcas.to_sql(
    'schools_mcas',
    schema='infrastructure',
    con=cnx,
    if_exists='append',
    index=False
) 

int_value_pairs.to_sql('int_value_pairs',schema='common',
                    con=cnx,if_exists='append',index=False)

#### Finances

In [ ]:
##finances
finance_extracts = {
    "nss": {
        "url" : 'netschoolspendingtrend.aspx',
        "ids" : {
            "year"   : "ctl00_ContentPlaceHolder1_ddYear",
            "button" : "btnViewReport"
        },
    },
    "ppx" : {
        "url" : 'ppx.aspx',
        "ids" : {
            "year"   : "ctl00_ContentPlaceHolder1_ddYear",
            "button" : "btnViewReport"
        },
    },
}

datasets = {}
int_value_pairs = pd.DataFrame()

for category in list(finance_extracts.keys())[1:]:

    driver = start_up(
        stub + finance_extracts[category]['url']
    )
    time.sleep(2)
    data = get_teacher_details(driver,finance_extracts[category]['ids'])
    if 'FTE Count' in data.columns:
        data = data[~pd.isnull(data['FTE Count'])]

    datasets[category], tmp_ivp =\
        transform_teacher_details(
           data.copy(),
           category = category
    )
    int_value_pairs = pd.concat([int_value_pairs,tmp_ivp])

    mask = datasets[category]['school_id']!=0
    datasets[category]=datasets[category][mask]
    
    datasets[category]['value']=\
        round(datasets[category]['value'].astype(str)\
              .str.replace("\$|\,","",regex=True)\
              .astype(float),0).astype(int)

    driver.close()


for key in datasets.keys():
    cnx.execute(table_create_schools.format(category=key))
    datasets[key].to_sql(
        'schools_'+key.lower(),
        schema='infrastructure',
        con=cnx,
        if_exists='append',
        index=False
    ) 

int_value_pairs.to_sql('int_value_pairs',schema='common',
                    con=cnx,if_exists='append',index=False)    

#### Teacher Characteristics

In [ ]:
## teachers staffing and salaries
int_value_pairs = pd.DataFrame()

teacher_extracts = {
    'teacher_salaries': {
        'url' : 'teachersalaries.aspx',
        'ids' : {
            'year':'ctl00_ContentPlaceHolder1_ddYear',
            'button':'btnViewReport',
        },
    },
    'teacher_race':{
        'url':'teacherbyracegender.aspx',
        'ids': {
            "year"   : "ctl00_ContentPlaceHolder1_ddYear",
            "button" : "ctl00_ContentPlaceHolder1_btnViewReport",
        },
    },
    'teacher_age':{
        'url':'agestaffing.aspx',
        'ids': {
            "year"   : "ctl00_ContentPlaceHolder1_ddYear",
            "button" : "btnViewReport",
        },
    },
    'teacher_program_area':{
        'url':'programareastaffing.aspx',
        'ids': {
            "year"   : "ddYear",
            "button" : "btnViewReport",
        },
    },
}

datasets = {}

for category in teacher_extracts.keys():

    driver = start_up(
        stub + teacher_extracts[category]['url']
    )
    time.sleep(2)
    data = get_teacher_details(driver,teacher_extracts[category]['ids'])
    if 'FTE Count' in data.columns:
        data = data[~pd.isnull(data['FTE Count'])]

    datasets[category], tmp_ivp =\
        transform_teacher_details(
           data.copy(),
           category = category
    )
    int_value_pairs = pd.concat([int_value_pairs,tmp_ivp])

    datasets[category]['value']=\
        round(datasets[category]['value'].astype(float),0).astype(int)

    driver.close()
    
##not sure how a blank year appeared
##remove state totals school_id=0
for key in datasets.keys():
    mask = (datasets[key].year!='')&\
            (datasets[key].school_id!=0)&\
                (~datasets[key].duplicated(['school_id','year',key]))
    datasets[key]=datasets[key][mask]
    


for key in datasets.keys():
    cnx.execute(table_create_teachers.format(category=key))
    datasets[key].to_sql(
        'schools_'+key.lower(),
        schema='infrastructure',
        con=cnx,
        if_exists='append',
        index=False
    ) 

int_value_pairs.to_sql('int_value_pairs',schema='common',
                    con=cnx,if_exists='append',index=False)

#### School Types

In [ ]:
url = 'schoolattendingchildren.aspx'
driver = start_up(stub+url)
time.sleep(2)

data = get_schools_types(driver)
driver.close()

schools_types, int_value_pairs = transform_schools_types(data)

cnx.execute(table_create_schools_types)

int_value_pairs.to_sql('int_value_pairs',schema='common',
                       con=cnx,if_exists='append',index=False)

schools_types.to_sql('schools_types',schema='infrastructure',
          con=cnx,if_exists='append',index=False)

#### Combine

In [ ]:
query = """
    select * from common.int_value_pairs;
"""
int_value_pairs = pd.read_sql_query(query,cnx)


all_df = pd.DataFrame()
cols=[]
query = """
    select * from infrastructure.schools_{tablename};
"""
for tablename in ['ppx','nss',
                  'teacher_age',
                  'teacher_salaries',
                  'teacher_program_area',
                  'teacher_race'
                 ]:
    df = pd.read_sql_query(query.format(tablename=tablename),cnx)    
    for col in df.columns:
        if col in int_value_pairs.item.unique():
            if col !='school_id':
                df = merge_int_value(df,int_value_pairs,col)
                df = df.rename(columns={col:'group'})
                for item in df.group.unique():
                    if item in cols:
                        print('dup',item)
                    cols.append(item)

                
            
    all_df = pd.concat([all_df,df])



## Analysis

In [ ]:
query = """
    select * from common.int_value_pairs;
"""
int_value_pairs = pd.read_sql_query(query,cnx)

query = """
    select m.*
    from infrastructure.schools_mcas m
    left join common.school_id_location lo on lo.school_id=m.school_id
    where lo.dor = 10
"""

mcas = pd.read_sql_query(query,cnx)
for col in mcas.columns:
    if col in int_value_pairs.item.unique():
        mcas = merge_int_value(mcas,int_value_pairs,col)

mcas

In [ ]:
mcas.pivot(columns=['mcas'],
           index=['school_id','year','mcas_subject','mcas_grade'],
          values=['value']).reset_index().replace(np.nan,None)

In [ ]:
def schools_pivot(df,category):
    
    if category == 'enrollment':
        index = ['school_id','year']
        columns = ['grade']
    elif category == 'mcas':
        index=['school_id','year','mcas_subject','mcas_grade']
        columns = ['mcas']
    else:
        index=['year']
        columns=[category]

    df = df.pivot(columns=columns,
                  values=['value'],index=index)\
        .reset_index()
    df.columns=df.columns.get_level_values(1)
    df.columns=index+list(df.columns[len(index):])
    df = df.sort_values(['year'],ascending=False).reset_index(drop=True).replace(np.nan,'').astype(str).replace({'\.0$':''},regex=True)
    if columns == ['grade']:
        cols = ['school_id','year',
                'PK','K',
                '1','2','3','4','5',
                '6','7','8',
                '9','10','11','12',
                'SP','Total']
        df = df[cols]
        
    return df

category = 'mcas'
query = """
    select x.*
    from infrastructure.schools_{category} x
    left join common.school_id_location lo on lo.school_id=x.school_id
    where lo.dor = 10
""".format(category=category)

df = pd.read_sql_query(query,cnx)
for col in df.columns:
    if col in int_value_pairs.item.unique():
        df = merge_int_value(df,int_value_pairs,col)


schools_pivot(df,category)

#### Misc

In [ ]:
url = 'https://gis-arlingtonma.opendata.arcgis.com/maps/689f7595316a4d7a84e8fc7b9602a945'
filename = 'ArlingtonMA_SchoolDistrict.geojson'
import geopandas as gpd
geo = gpd.read_file(filename)